# Фильтрация и агрегатные функции

## РАБОТА С ОБЪЕДИНЁННЫМИ ТАБЛИЦАМИ

### ✍ Соединять таблицы мы научились, теперь давайте научимся получать необходимые данные из нескольких таблиц ↓

### Принцип построения запроса и порядок операторов такой же, как и с обычной таблицей.

### Вспомним его:

* SELECT... 
* FROM... 
* WHERE... 
* GROUP BY... 
* ORDER BY... 
* LIMIT...

## ФИЛЬТРАЦИЯ ДАННЫХ

### К соединённым таблицам применимы функции фильтрации данных.

### Например, можно вывести id матчей, в которых команда Arsenal была гостевой.

In [ ]:
'''
SELECT 
    m.id /*столбец id таблицы m*/
FROM
    sql.teams t /*таблица teams с алиасом t*/
    JOIN sql.matches m ON m.away_team_api_id = t.api_id /*оператор соединения таблиц; таблица matches с алиасом m; условие: away_team_api_id таблицы m равен api_id таблицы t*/
WHERE long_name = 'Arsenal' /*long_name таблицы teams имеет значение Arsenal*/
'''

### Принципиальное отличие фильтрации данных по соединённым таблицам от аналогичного действия по одиночным таблицам заключается в том, что, фильтруя записи одной таблицы, мы также будем фильтровать и записи другой таблицы, поскольку соединённые на уровне запроса таблицы по сути являются единой таблицей.

### Например, результат запроса

### SELECT 
### 	m.id id_1,
### 	m.season,
### 	t.id id_2,
### 	t.long_name
### FROM
### 	sql.teams t
### JOIN sql.matches m ON m.away_team_api_id = t.api_id

### можно разделить на две разные части

![Alt text](https://lms-cdn.skillfactory.ru/assets/courseware/v1/7281095f07ca452dfb5963c7ea912fe7/asset-v1%3ASkillFactory%2BDSPR-2.0%2B14JULY2021%2Btype%40asset%2Bblock/dst3-u2-md3_4_1.png)

### Одна часть — таблица matches с алиасом m, вторая — teams с алиасом t, но после соединения они являются одной таблицей.

### Таким образом, если вы отфильтруете данные по одной части таблицы, то другая, соединённая, часть пропадёт вместе с ней.

In [ ]:
# 4.1 
'''
select
    t.long_name long_name,
    m.home_team_goals home_goal,
    m.away_team_goals away_goal
from sql.matches m 
    join sql.teams t on m.home_team_api_id = t.api_id 
where t.short_name = 'GEN'
group by m.id, long_name, home_goal, away_goal
order by m.id
'''

### Также мы можем отфильтровать записи сразу по двум таблицам.

### Например, можно оставить только записи, в которых короткое название домашней команды GEN и матчи сезона 2008/2009.



In [ ]:
'''
SELECT * /*выбор всех полей*/
FROM    
    sql.matches m /*таблица matches с алиасом m*/
    JOIN sql.teams t on t.api_id = m.home_team_api_id /*оператор соединения таблиц; таблица teams с алиасом t; условие: home_team_api_id таблицы m равен api_id таблицы t*/
WHERE
    t.short_name = 'GEN' /*столбец short_name таблицы t имеет значение GEN*/
    AND m.season = '2008/2009' /*столбец season таблицы m имеет значение 2008/2009*/
'''

In [ ]:
# 4.2
'''
select
  m.id,
  h.short_name home_short,
  a.short_name away_short
from sql.matches m
  join sql.teams h on m.home_team_api_id = h.api_id
  join sql.teams a on m.away_team_api_id = a.api_id
where m.season='2011/2012' and (h.long_name = 'Liverpool' or a.long_name = 'Liverpool')
group by m.id, home_short, away_short
'''

## АГРЕГАЦИЯ ДАННЫХ

### К соединённым таблицам также применимы любые агрегатные функции — самые важные функции для анализа данных.

### Например, мы можем вывести сумму голов по командам для матчей, где команда выступала в гостях.



In [ ]:
'''
SELECT
    t.long_name, /*столбец long_name таблицы t*/
    SUM(m.home_team_goals) + SUM(m.away_team_goals) match_goals /*функция суммирования; столбец home_team_goals таблицы m; функция суммирования; столбец away_team_goals таблицы m; новое название столбца*/
FROM
    sql.matches m /*таблица matches с алиасом m*/
    JOIN sql.teams t ON m.away_team_api_id = t.api_id /*оператор соединения таблиц; таблица teams с алиасом t; условие: away_team_api_id таблицы m равен api_id таблицы t*/
GROUP BY t.id /*группировка по столбцу id таблицы t*/
'''

## Обратите внимание! В данном запросе была использована группировка по столбцу id таблицы teams, хотя этот столбец не выводится в запросе. Это необходимо для того, чтобы команды с одинаковым названием, если такие найдутся, не группировались между собой. Группировка по названию команды в данном запросе будет неверной, так как есть несколько команд с одинаковым полным названием — мы говорили об этом в начале модуля.

### Также, применяя агрегатные функции к соединённым таблицам, обращайте внимание на указание алиасов (или таблиц) при группировке и указании столбцов агрегатных функций. В нашей соединённой таблице есть два столбца с названием id, и если бы мы сформировали запрос без указания таблицы, как указано ниже, то...

In [ ]:
'''
SELECT
    t.long_name, /*столбец long_name таблицы t*/
    SUM(m.home_team_goals) + SUM(m.away_team_goals) match_goals /*функция суммирования; столбец home_team_goals таблицы m; функция суммирования; столбец away_team_goals таблицы m; новое название столбца*/
FROM
    sql.matches m /*таблица matches с алиасом m*/
    JOIN sql.teams t ON m.away_team_api_id = t.api_id /*оператор соединения таблиц; таблица teams с алиасом t; условие: away_team_api_id таблицы m равен api_id таблицы t*/
GROUP BY id /*группировка по столбцу id, но не указано, какой таблицы*/
'''

### система выдала бы уже знакомую нам ошибку: "... column "id" is ambiguous ...".

### Мы можем использовать оператор HAVING для фильтрации сгруппированных данных.

### Поставим задачу — вывести таблицу с суммарным количеством забитых голов в матчах по командам и сезонам для команд, в которых суммарное количество голов в матчах сезона больше 100.

### Для начала посчитаем общее количество голов в матчах по сезонам.

In [ ]:
'''
SELECT
    m.season, /*столбец season таблицы m*/
    SUM(m.home_team_goals) + SUM(m.away_team_goals) total_goals /*функция суммирования; столбец home_team_goals таблицы m; функция суммирования; столбец away_team_goals таблицы m; новое название столбца*/
FROM sql.matches m /*таблица matches с алиасом m*/
GROUP BY m.season /*группировка по столбцу season таблицы m*/
'''

### Затем добавим таблицу с командами, группировку по командам и условие фильтрации.

In [ ]:
'''
SELECT
    m.season, /*столбец season таблицы m*/
    t.long_name, /*столбец long_name таблицы t*/
    SUM(m.home_team_goals) + SUM(m.away_team_goals) total_goals /*функция суммирования; столбец home_team_goals таблицы m; функция суммирования; 
    столбец away_team_goals таблицы m; новое название столбца*/
FROM sql.matches m /*таблица matches с алиасом m*/
JOIN sql.teams t ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id /*оператор соединения таблиц; таблица teams с алиасом t; 
условие: home_team_api_id таблицы m равен api_id таблицы t или away_team_api_id таблицы m равен api_id таблицы t*/
GROUP BY m.season, t.id /*группировка по столбцам season таблицы m и id таблицы t*/
HAVING SUM(m.home_team_goals) + SUM(m.away_team_goals) > 100 /*оператор фильтрации сгруппированных данных; функция суммирования; 
home_team_goals таблицы m; функция суммирования; away_team_goals таблицы m; больше 100*/
'''